In [2]:
from loader import Loader
from sklearn import svm
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt 
import numpy as np

def flatten_image (X, length):
    X_flatten = np.empty([length,16*15])
    for i in range (length):
        X_flatten[i] = X[i].flatten()
    return X_flatten

def add_noise (X, intensity):
    examples, imsize = np.shape(X)
    for i in range (examples):
        noise = np.random.normal(0,1, imsize)
        X[i] += intensity*noise
    return X

def cross_val_with_noise (X,y,k,intensity,clf):
    scores = np.zeros(k)
    fold_len = int(len(X)/k)
    X_test = X[0:fold_len, :]
    y_test = y[0:fold_len]
    X_train = X[fold_len:, :]
    y_train = y[fold_len:]
    
    X_train = add_noise(X_train, intensity)
    
    for i in range(k):
        clf.fit(X_train,y_train)
        y_predict = clf.predict(X_test)
        mismatch = 0
        for j in range(len(y_test)):
            if y_test[j] != y_predict[j]:
                mismatch+=1
        scores[i] = (1-mismatch/len(y_test))*100
        X_test = X[i*fold_len:(i+1)*fold_len, :]
        
        y_test = y[i*fold_len:(i+1)*fold_len]
        X_train = np.append(X[0:i*fold_len, :], X[(i+1)*fold_len:, :], axis = 0)
        y_train = np.append(y[0:i*fold_len], y[(i+1)*fold_len:], axis = 0)
        
        X_train = add_noise(X_train, intensity)
        
    return scores;

In [3]:
dataset = Loader()
del dataset
dataset = Loader()

In [27]:
noise_opt = 0.1
c_opt = 4.0
dataset.pca(n_comp=75)

accuracy = 0
for n in range(500):
    
    X, y = dataset.getNoisySet(intensity = noise_opt, pca=True, set="train")
    clf = svm.SVC(C=c_opt)
    clf.fit(X,y)
    X_test, y_test = dataset.getWholeTestSet(pca=True)
    y_predict = clf.predict(X_test)
    mismatch = 0
    for i in range(len(y_test)):
        if y_test[i] != y_predict[i]:
            mismatch+=1
    accuracy += (1-mismatch/len(y_test))/5
print ("Test accuracy:", accuracy , "%")

Test accuracy: 97.83540000000022 %


In [ ]:
dataset.pca(n_comp=75)
acc_matrix = np.zeros([9,7])
num = 100
for N in range(num):
    i = 0
    for C in [1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0, 7.5, 10.0]:
    
        j = 0
        for noise_int in [0.0,0.1,0.2,0.3,0.4,0.5,0.6]:
            X, y = dataset.getWholeTrainSet(pca=True, shuffle=True)
            clf = svm.SVC(C=C)
            scores = cross_val_with_noise(X,y,10,noise_int,clf)
            #print(scores)
            acc_matrix[i,j] += np.mean(scores)/num
            j+=1
        i+=1
    
print(acc_matrix)

In [ ]:
accs = np.zeros(len(crange))
i = 0
dataset.pca(n_comp=75)
X, y = dataset.getWholeTrainSet(shuffle=True, pca=True)
X_test, y_test = dataset.getWholeTestSet(shuffle=True, pca=True)
for C in crange:
    clf = svm.SVC(C=C)
    clf.fit(X,y)
    y_predict = clf.predict(X_test)
    mismatch = 0
    for j in range(len(y_test)):
        if y_test[j] != y_predict[j]:
            mismatch+=1
    accuracy = (1-mismatch/len(y_test))
    accs[i] = accuracy*100
    i+=1
print(accs)

In [31]:
print(SVM_noise())


[97.7 97.7 97.8 97.9 97.8 97.6 97.9 97.9 97.9 97.9 97.6]
